![Gambar](AB Test.png)
# A/B Testing Case
### Oleh : Topik Zulkarnain
## Introduction A/B Testing
A/B Testing adalah teknik yang digunakan untuk menentukan apakah suatu rancangan baru dari suatu hal dapat membawa improvement berdasarkan metrik yang dipilih. Pada notebook ini akan dilakukan A/B Testing terhadap kasus membandingkan dua promosi suatu perusahaan e-commerce , yaitu membandingkan promosi yang sudah ada (A) dan promosi yang baru (B).
Metrik yang digunakan oleh perusahaan adalah :
* Rataan aktifitas (dalam waktu) pengguna aplikasi/web.
* Conversion Rate, yang didefinisikan proporsi orang yang akhirnya membeli barang setelah melihat aplikasi/web.

Pembagian traffic antara promosi A dan B adalah 60% - 40%

## Data

In [31]:
import pandas as pd
data_dict = {
    'Jumlah Pengguna':[6000,4000],
    'Rataan (waktu)': [60,62],
    'std (waktu)' : [40,45],
    'Conversion Rate' : [1.5,2],
    'Index' : ['A','B']
}

In [32]:
data = pd.DataFrame(data_dict, columns = ['Jumlah Pengguna', 'Rataan (waktu)', 'std (waktu)', 'Conversion Rate'], index=['A','B'])

In [33]:
data

,Jumlah Pengguna,Rataan (waktu),std (waktu),Conversion Rate
A,6000,60,40,1.5
B,4000,62,45,2.0


Berdasarkan data, dapat dilihat bahwa untuk promosi B terlihat para pelanggan lebih lama dalam melihat web/aplikasi, namun conversion ratenya lebbih tinggi dibandingkan dengan promosi A. Untuk membuktikan apakah promosi B benar benar memberikan impact business yang lebih baik, maka diperlukannya A/B Testing. Berikut adalah hal yang perlu diperhatikan saat menggunakan A/B Testing :
1. Membandingkan parameter dari distribusi 2 populasi nya ataupun membandingkan distribusinya terhadap populasinya sendiri
2. Membangun hipotesis null dan hipotesis alternatif untuk pengambilan kesimpulan

Test statistik yang digunakan pada kasus ini adalah :
* Uji-Z untuk melihat apakah benar promosi B lebih membuat pelanggan menghabiskan waktunya melihat web
* Uji-Chi Square untuk melihat apakah benar promosi B membuat impact pembelian lebih banyak dibandingkan A

### 1. Uji-Z untuk Rataan Waktu

Hipotesis yang akan digunakan adalah :
* H0 = rataan waktu antara dua promosi adalah sama
* Ha = rataan waktu promosi B lebih lama 

In [35]:
import numpy as np
from scipy.stats import norm

mu_B = 62
mu_A = 60

std_B = 45
std_A = 40

n_B = 4000
n_A = 6000

Z = (mu_B - mu_A)/np.sqrt(std_B**2/n_B + std_A**2/n_A)
pvalue = norm.sf(Z)

print("Z-score: {0}\np-value: {1}".format(Z,pvalue))

Z-score: 2.2749070654279993
p-value: 0.011455752709549046


dengan menggunakan alpha 5%, didapatkan p-value < alpha. Maka disimpulkan H0 ditolak yang berarti rataan waktu promosi B lebih lama

## 2. Uji Chi-Square untuk Conversion Rate
Hipotesis yang digunakan adalah :
* H0 : conversion rate pada dua promosi adalah sama
* H1 : conversion rate pada promosi B lebih besar

data yang digunakan berdasarkan conversion rate adalah sebagai berikut:

In [36]:
import pandas as pd
data_dict = {
    'Jumlah Pengguna':[6000,4000],
    'Converted': [90,80],
    'Non Converted' : [5910,3920],
    'Index' : ['A','B']
}

In [37]:
data_conversion = data = pd.DataFrame(data_dict, columns = ['Jumlah Pengguna', 'Converted', 'Non Converted'], index=['A','B'])

In [38]:
data_conversion

,Jumlah Pengguna,Converted,Non Converted
A,6000,90,5910
B,4000,80,3920


Pada hipotesis H0, conversion promosi A dan B terdistribusi dengan sama yaitu terdistribusi binomial. Maka harus digunakan estimasi untuk menghitung conversion rate dibawah hipotesis H0, dan didapat hasilnya sebagai berikut :

In [39]:
import pandas as pd
data_dict = {
    'Jumlah Pengguna':[6000,4000],
    'Converted': [102,68],
    'Non Converted' : [5898,3932],
    'Index' : ['A','B']
}

In [40]:
data_conversion = data = pd.DataFrame(data_dict, columns = ['Jumlah Pengguna', 'Converted', 'Non Converted'], index=['A','B'])

In [41]:
data_conversion

,Jumlah Pengguna,Converted,Non Converted
A,6000,102,5898
B,4000,68,3932


In [42]:
from scipy.stats import chi2
import numpy as np

T = np.array([102, 68, 5898, 3932])
O = np.array([90, 80, 5910, 3920])

D = np.sum(np.square(T-O)/T)

pvalue = chi2.sf(D, df=1)

print("distance d: {0}\np-value: {1}".format(D,pvalue))

distance d: 3.590449404583807
p-value: 0.05811252579106675


didapatkan p-value>alpha, maka dapat disimpulkan bahwa conversion rate antara promosi A dan B adalah sama. Namun, jika dilihat nilai p-value sangat tipis dengan nilai alpha. Oleh karena itu, diperlukan analisis lain untuk mengecek apakah terdapat uji lain yang bisa menurunkan p-value lebih sedikit lagi sehingga dapat disimpulkan bahwa conversion rate promosi B lebih besar dan akhirnya pun didapatkan insight bahwa promosi B bisa dibilang lebih baik dari pada promosi A. Uji yang akan dilakukan adalah mencoba melakukan Z-Test untuk conversion rate

## 3. Uji-Z untuk Conversion Rate

In [43]:
import numpy as np
from scipy.stats import norm

mu_B = 0.02
mu_A = 0.015

var_B = mu_B * (1-mu_B)
var_A = mu_A * (1-mu_A)

n_B = 4000
n_A = 6000

Z = (mu_B - mu_A)/np.sqrt(var_B/n_B + var_A/n_A)
pvalue = norm.sf(Z)

print("Z-score: {0}\np-value: {1}".format(Z,pvalue))

Z-score: 1.8427115179918694
p-value: 0.03268557071858785


dengan menggunakan uji-Z untuk konversion rate ternyata p-value<alpha sehingga dapat disimpulkan bahwa, conversion rate untuk promosi B lebih baik dari pada promosi A, sehingga pada nantinya perusahaan bisa mengembangkan promosi B untuk mendapatkan keuntungan yang lebih besar lagi bagi bisnisnya